# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 6 2022 2:40PM,241225,10,MSP211907,"Methapharm, Inc.",Released
1,Jul 6 2022 2:40PM,241225,10,MSP211908,"Methapharm, Inc.",Released
2,Jul 6 2022 2:40PM,241225,10,MSP211909,"Methapharm, Inc.",Released
3,Jul 6 2022 2:40PM,241225,10,MSP211910,"Methapharm, Inc.",Released
4,Jul 6 2022 2:40PM,241225,10,MSP211911,"Methapharm, Inc.",Released
5,Jul 6 2022 2:40PM,241225,10,MSP211912,"Methapharm, Inc.",Released
6,Jul 6 2022 2:40PM,241225,10,MSP211915,"Methapharm, Inc.",Released
7,Jul 6 2022 2:40PM,241225,10,MSP211916,"Methapharm, Inc.",Released
8,Jul 6 2022 2:40PM,241225,10,MSP211917,"Methapharm, Inc.",Released
9,Jul 6 2022 2:40PM,241225,10,MSP211918,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
55,241222,Released,11
56,241223,Released,9
57,241224,Released,59
58,241225,Released,20
59,241226,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241222,NaN,NaN,11.0
241223,NaN,NaN,9.0
241224,NaN,NaN,59.0
241225,NaN,NaN,20.0
241226,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241095,0.0,7.0,1.0
241101,0.0,4.0,0.0
241118,0.0,3.0,5.0
241119,0.0,1.0,0.0
241120,3.0,1.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241095,0,7,1
241101,0,4,0
241118,0,3,5
241119,0,1,0
241120,3,1,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241095,0,7,1
1,241101,0,4,0
2,241118,0,3,5
3,241119,0,1,0
4,241120,3,1,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241095,,7,1
1,241101,,4,
2,241118,,3,5
3,241119,,1,
4,241120,3,1,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 6 2022 2:40PM,241225,10,"Methapharm, Inc."
20,Jul 6 2022 2:40PM,241226,10,"Methapharm, Inc."
22,Jul 6 2022 2:25PM,241224,19,"AdvaGen Pharma, Ltd"
81,Jul 6 2022 2:17PM,241223,10,ISDIN Corporation
90,Jul 6 2022 2:17PM,241219,10,ISDIN Corporation
118,Jul 6 2022 2:14PM,241222,19,"AdvaGen Pharma, Ltd"
129,Jul 6 2022 2:08PM,241220,10,"Citieffe, Inc."
130,Jul 6 2022 1:47PM,241202,20,"ACI Healthcare USA, Inc."
135,Jul 6 2022 1:23PM,241217,20,Alumier Labs Inc.
151,Jul 6 2022 1:01PM,241215,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 6 2022 2:40PM,241225,10,"Methapharm, Inc.",,,20
1,Jul 6 2022 2:40PM,241226,10,"Methapharm, Inc.",,,2
2,Jul 6 2022 2:25PM,241224,19,"AdvaGen Pharma, Ltd",,,59
3,Jul 6 2022 2:17PM,241223,10,ISDIN Corporation,,,9
4,Jul 6 2022 2:17PM,241219,10,ISDIN Corporation,,,28
5,Jul 6 2022 2:14PM,241222,19,"AdvaGen Pharma, Ltd",,,11
6,Jul 6 2022 2:08PM,241220,10,"Citieffe, Inc.",,,1
7,Jul 6 2022 1:47PM,241202,20,"ACI Healthcare USA, Inc.",,3,2
8,Jul 6 2022 1:23PM,241217,20,Alumier Labs Inc.,,,16
9,Jul 6 2022 1:01PM,241215,19,ACG North America LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 2:40PM,241225,10,"Methapharm, Inc.",20,,
1,Jul 6 2022 2:40PM,241226,10,"Methapharm, Inc.",2,,
2,Jul 6 2022 2:25PM,241224,19,"AdvaGen Pharma, Ltd",59,,
3,Jul 6 2022 2:17PM,241223,10,ISDIN Corporation,9,,
4,Jul 6 2022 2:17PM,241219,10,ISDIN Corporation,28,,
5,Jul 6 2022 2:14PM,241222,19,"AdvaGen Pharma, Ltd",11,,
6,Jul 6 2022 2:08PM,241220,10,"Citieffe, Inc.",1,,
7,Jul 6 2022 1:47PM,241202,20,"ACI Healthcare USA, Inc.",2,3,
8,Jul 6 2022 1:23PM,241217,20,Alumier Labs Inc.,16,,
9,Jul 6 2022 1:01PM,241215,19,ACG North America LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 2:40PM,241225,10,"Methapharm, Inc.",20,,
1,Jul 6 2022 2:40PM,241226,10,"Methapharm, Inc.",2,,
2,Jul 6 2022 2:25PM,241224,19,"AdvaGen Pharma, Ltd",59,,
3,Jul 6 2022 2:17PM,241223,10,ISDIN Corporation,9,,
4,Jul 6 2022 2:17PM,241219,10,ISDIN Corporation,28,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 2:40PM,241225,10,"Methapharm, Inc.",20.0,NaN,NaN
1,Jul 6 2022 2:40PM,241226,10,"Methapharm, Inc.",2.0,NaN,NaN
2,Jul 6 2022 2:25PM,241224,19,"AdvaGen Pharma, Ltd",59.0,NaN,NaN
3,Jul 6 2022 2:17PM,241223,10,ISDIN Corporation,9.0,NaN,NaN
4,Jul 6 2022 2:17PM,241219,10,ISDIN Corporation,28.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 2:40PM,241225,10,"Methapharm, Inc.",20.0,0.0,0.0
1,Jul 6 2022 2:40PM,241226,10,"Methapharm, Inc.",2.0,0.0,0.0
2,Jul 6 2022 2:25PM,241224,19,"AdvaGen Pharma, Ltd",59.0,0.0,0.0
3,Jul 6 2022 2:17PM,241223,10,ISDIN Corporation,9.0,0.0,0.0
4,Jul 6 2022 2:17PM,241219,10,ISDIN Corporation,28.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,8682506,546.0,10.0,0.0
12,723561,13.0,1.0,0.0
16,1446838,3.0,12.0,0.0
19,964781,79.0,1.0,3.0
20,964789,63.0,3.0,0.0
21,241183,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,8682506,546.0,10.0,0.0
1,12,723561,13.0,1.0,0.0
2,16,1446838,3.0,12.0,0.0
3,19,964781,79.0,1.0,3.0
4,20,964789,63.0,3.0,0.0
5,21,241183,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,546.0,10.0,0.0
1,12,13.0,1.0,0.0
2,16,3.0,12.0,0.0
3,19,79.0,1.0,3.0
4,20,63.0,3.0,0.0
5,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,546.0
1,12,Released,13.0
2,16,Released,3.0
3,19,Released,79.0
4,20,Released,63.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,3.0,0.0,0.0
Executing,10.0,1.0,12.0,1.0,3.0,1.0
Released,546.0,13.0,3.0,79.0,63.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,3.0,0.0,0.0
1,Executing,10.0,1.0,12.0,1.0,3.0,1.0
2,Released,546.0,13.0,3.0,79.0,63.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,3.0,0.0,0.0
1,Executing,10.0,1.0,12.0,1.0,3.0,1.0
2,Released,546.0,13.0,3.0,79.0,63.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()